#Import packages and libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/aipi540_final_proj/

/content/drive/MyDrive/aipi540_final_proj


In [3]:
import sys
sys.path.append('drive/MyDrive/aipi540_final_proj')

In [6]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords 
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.stem import WordNetLemmatizer
#import spacy #for faster tokenization and lemmatization

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import re
import string

import functions

%pylab inline

Populating the interactive namespace from numpy and matplotlib


#Load all recipe datasets

Food Network

In [7]:
data_fn = pd.read_json("./recipes_raw_nosource_fn.json")

In [8]:
data_fn = data_fn.T
display(data_fn)

,instructions,ingredients,title,picture_link
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,Toss ingredients lightly and spoon into a butt...,"[1/2 cup celery, finely chopped, 1 small green...",Grammie Hamblet's Deviled Crab,None
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,Watch how to make this recipe.\nSprinkle the s...,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Infineon Raceway Baked Beans,Ja5uaD8Q7m7vvtWwk2.48dr1eCre/qi
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,"In a large saucepan, let the beans soak in eno...","[1 1/2 cups dried black beans, picked over and...",Southwestern Black Bean Dip,None
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,Watch how to make this recipe.\nPreheat the ov...,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Sour Cream Noodle Bake,nm/WxalB.VjEZSa0iX9RuZ8xI51Y7bS
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,Special equipment: sushi mat\nCook the brown r...,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Sushi Renovation,None
...,...,...,...,...
4bfMWxlbKhx/McJq/89k0SBdw.VvAzW,Watch how to make this recipe.\nPreheat a gril...,"[4 ears fresh corn, 2 heads Belgian endive, 2 ...",Summer Corn Salad,G9XPV1I0nULIbwS76f..0AIq2DsfI3u
T8lWBA1fcVdjxhMSWuoAbGoy5Lj.A8m,Preheat the broiler. Cut the tomatoes in 1/2 c...,"[4 large plum tomatoes, Salt and sugar, 1 1/2 ...",Zucchini Stuffed Tomatoes,nA7V5iQqaFcIEQQ0/MKqVdUioieXjPu
f/coffo2TMs2J2gq5nTOUIqH2TRAkui,Heat the oil and butter in a large skillet ove...,"[3 tablespoons olive oil, 2 tablespoons unsalt...",Pepper Pasta Quick Cook,None
q3aDJc4zoEF5QT4e7Mn.ieQwV.DyHwS,Preheat oven to 350 degrees. On the top half o...,"[8 ounces butter, 8 ounces bittersweet chocola...",Chocolate Cake with Armagnac Ice Cream,None


In [9]:
data_fn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60039 entries, p3pKOD6jIHEcjf20CCXohP8uqkG5dGi to 7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   instructions  59628 non-null  object
 1   ingredients   59802 non-null  object
 2   title         59750 non-null  object
 3   picture_link  30952 non-null  object
dtypes: object(4)
memory usage: 4.3+ MB


Epicurious

In [10]:
data_epi = pd.read_json("./recipes_raw_nosource_epi.json")

In [11]:
data_epi = data_epi.T
display(data_epi)

,ingredients,picture_link,instructions,title
05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby,"[12 egg whites, 12 egg yolks, 1 1/2 cups sugar...",None,"Beat the egg whites until stiff, gradually add...",Christmas Eggnog
mF5SZmoqxF4WtIlhLRvzuKk.z6s7P2S,"[18 fresh chestnuts, 2 1/2 pounds veal stew me...",None,Preheat oven to 400°F. Using small sharp knife...,"Veal, Carrot and Chestnut Ragoût"
oQV5D7cVbCFwmrDs3pBUv2y.AG0WV26,"[2 tablespoons unsalted butter, softened, 4 or...",3xjktRst3I5lDZ2Z5kTOtqQyzZFFN9u,Preheat the oven to 350°F. Spread the softened...,Caramelized Bread Pudding with Chocolate and C...
Z9seBJWaB5NkSp4DQHDnCAUBTwov/1u,"[3/4 pound Stilton, crumbled (about 3 cups) an...",None,"In a food processor blend the Stilton, the cre...",Sherried Stilton and Green Peppercorn Spread
bB3GxoAplVZeoX3fzWNWyeECtQFxw6G,"[2 cups (about 9 1/2 ounces) whole almonds, to...",None,Position rack in center of oven and preheat to...,Almond-Chocolate Macaroons
...,...,...,...,...
lgygAJbdT1RNgrOYYveXtW4Ze9GtcZK,"[5 cups (about) canned beef broth, 1 cup lenti...",None,Bring 4 cups broth and lentils to boil in larg...,Spinach and Lentil Soup
EgIvlp1EfF4qTsnSI3v7ViZF1b3f2qC,"[1/2 teaspoon turmeric, 2 cups water, 3/4 poun...",16DvpBiksnvbWPCSdZUcPY3oFvgo0py,"Stir together turmeric, 1 1/2 cups water, and ...",Fresh Cheese with Spinach
aCaoP.P8A1h6ALQmBnw3ypvYfRt/zZe,"[1 medium jalapeño, thinly sliced, 2 garlic cl...",m6IALrfAOr5KDVpqKcvUNpa7VermSIq,"Marinate and cook the steak: Combine jalapeño,...",Jalapeño and Lime–Marinated Skirt Steak Tacos
fGxd/ZNUQcXxxqaVgEtMJmsNKLbugAe,"[1 1/2 cups almond flour or almond meal, 1/2 c...",wq1Cqtl16W8QpuFaXrDdB2kGsUn9Ku6,Make the cakes: Preheat oven to 350°F. Line th...,Semolina–Lemon Syrup Cakes


In [12]:
data_epi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25323 entries, 05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby to qznqHiNpq0AB1AYn002A2HvaEFnN0lq
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ingredients   25323 non-null  object
 1   picture_link  12119 non-null  object
 2   instructions  25323 non-null  object
 3   title         25323 non-null  object
dtypes: object(4)
memory usage: 2.0+ MB


Allrecipes

In [13]:
data_ar = pd.read_json("./recipes_raw_nosource_ar.json")

In [14]:
data_ar = data_ar.T
display(data_ar)

,title,ingredients,instructions,picture_link
rmK12Uau.ntP510KeImX506H6Mr6jTu,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe
clyYQv.CplpwJtjNaFGhx0VilNYqRxu,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56
BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi
N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm
...,...,...,...,...
gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,Thai-Indian Veggie Soup,"[2 teaspoons olive oil ADVERTISEMENT, 1/4 cup ...",Heat oil in a large pot over medium heat. Add ...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa
VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Coconut Milk-Free Panang Curry Chicken,"[2 cups light cream ADVERTISEMENT, 1/4 teaspoo...",Heat cream and coconut extract in a skillet or...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa
Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,Cooked Cold Salad,"[3 tablespoons bacon grease ADVERTISEMENT, 2 c...",Heat bacon grease in a skillet over medium-hig...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa
ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,Easy Eggnog Creme Brulee,"[4 egg yolks ADVERTISEMENT, 1 tablespoon white...",Preheat oven to 350 degrees F (175 degrees C)....,qE58a7Z1Au0GXvPO188iHZZVqna9hLa


In [15]:
data_ar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39802 entries, rmK12Uau.ntP510KeImX506H6Mr6jTu to 2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         39522 non-null  object
 1   ingredients   39522 non-null  object
 2   instructions  39522 non-null  object
 3   picture_link  39522 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


# Data Cleaning

Food Network

In [16]:
data_fn_cln = data_fn.loc[:, ["title", "ingredients", "instructions"]]

In [17]:
data_fn_cln = data_fn_cln.dropna(how='any')
display(data_fn_cln)
data_fn_cln.info()

,title,ingredients,instructions
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,Grammie Hamblet's Deviled Crab,"[1/2 cup celery, finely chopped, 1 small green...",Toss ingredients lightly and spoon into a butt...
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,Infineon Raceway Baked Beans,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Watch how to make this recipe.\nSprinkle the s...
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,Southwestern Black Bean Dip,"[1 1/2 cups dried black beans, picked over and...","In a large saucepan, let the beans soak in eno..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,Sour Cream Noodle Bake,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Watch how to make this recipe.\nPreheat the ov...
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,Sushi Renovation,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Special equipment: sushi mat\nCook the brown r...
...,...,...,...
4bfMWxlbKhx/McJq/89k0SBdw.VvAzW,Summer Corn Salad,"[4 ears fresh corn, 2 heads Belgian endive, 2 ...",Watch how to make this recipe.\nPreheat a gril...
T8lWBA1fcVdjxhMSWuoAbGoy5Lj.A8m,Zucchini Stuffed Tomatoes,"[4 large plum tomatoes, Salt and sugar, 1 1/2 ...",Preheat the broiler. Cut the tomatoes in 1/2 c...
f/coffo2TMs2J2gq5nTOUIqH2TRAkui,Pepper Pasta Quick Cook,"[3 tablespoons olive oil, 2 tablespoons unsalt...",Heat the oil and butter in a large skillet ove...
q3aDJc4zoEF5QT4e7Mn.ieQwV.DyHwS,Chocolate Cake with Armagnac Ice Cream,"[8 ounces butter, 8 ounces bittersweet chocola...",Preheat oven to 350 degrees. On the top half o...


<class 'pandas.core.frame.DataFrame'>
Index: 59628 entries, p3pKOD6jIHEcjf20CCXohP8uqkG5dGi to 7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         59628 non-null  object
 1   ingredients   59628 non-null  object
 2   instructions  59628 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


Epicurious

In [18]:
data_epi_cln = data_epi.loc[:, ["title", "ingredients", "instructions"]]

In [19]:
display(data_epi_cln)
data_epi_cln.info()

,title,ingredients,instructions
05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby,Christmas Eggnog,"[12 egg whites, 12 egg yolks, 1 1/2 cups sugar...","Beat the egg whites until stiff, gradually add..."
mF5SZmoqxF4WtIlhLRvzuKk.z6s7P2S,"Veal, Carrot and Chestnut Ragoût","[18 fresh chestnuts, 2 1/2 pounds veal stew me...",Preheat oven to 400°F. Using small sharp knife...
oQV5D7cVbCFwmrDs3pBUv2y.AG0WV26,Caramelized Bread Pudding with Chocolate and C...,"[2 tablespoons unsalted butter, softened, 4 or...",Preheat the oven to 350°F. Spread the softened...
Z9seBJWaB5NkSp4DQHDnCAUBTwov/1u,Sherried Stilton and Green Peppercorn Spread,"[3/4 pound Stilton, crumbled (about 3 cups) an...","In a food processor blend the Stilton, the cre..."
bB3GxoAplVZeoX3fzWNWyeECtQFxw6G,Almond-Chocolate Macaroons,"[2 cups (about 9 1/2 ounces) whole almonds, to...",Position rack in center of oven and preheat to...
...,...,...,...
lgygAJbdT1RNgrOYYveXtW4Ze9GtcZK,Spinach and Lentil Soup,"[5 cups (about) canned beef broth, 1 cup lenti...",Bring 4 cups broth and lentils to boil in larg...
EgIvlp1EfF4qTsnSI3v7ViZF1b3f2qC,Fresh Cheese with Spinach,"[1/2 teaspoon turmeric, 2 cups water, 3/4 poun...","Stir together turmeric, 1 1/2 cups water, and ..."
aCaoP.P8A1h6ALQmBnw3ypvYfRt/zZe,Jalapeño and Lime–Marinated Skirt Steak Tacos,"[1 medium jalapeño, thinly sliced, 2 garlic cl...","Marinate and cook the steak: Combine jalapeño,..."
fGxd/ZNUQcXxxqaVgEtMJmsNKLbugAe,Semolina–Lemon Syrup Cakes,"[1 1/2 cups almond flour or almond meal, 1/2 c...",Make the cakes: Preheat oven to 350°F. Line th...


<class 'pandas.core.frame.DataFrame'>
Index: 25323 entries, 05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby to qznqHiNpq0AB1AYn002A2HvaEFnN0lq
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         25323 non-null  object
 1   ingredients   25323 non-null  object
 2   instructions  25323 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


Allrecipes

In [20]:
data_ar_cln = data_ar.loc[:, ["title", "ingredients", "instructions"]]

In [21]:
data_ar_cln = data_ar_cln.dropna(how='any')
display(data_ar_cln)
data_ar_cln.info()

,title,ingredients,instructions
rmK12Uau.ntP510KeImX506H6Mr6jTu,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ..."
5ZpZE8hSVdPk2ZXo1mZTyoPWJRSCPSm,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ..."
clyYQv.CplpwJtjNaFGhx0VilNYqRxu,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....
BmqFAmCrDHiKNwX.IQzb0U/v0mLlxFu,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....
N.jCksRjB4MFwbgPFQU8Kg.yF.XCtOi,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...
...,...,...,...
gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,Thai-Indian Veggie Soup,"[2 teaspoons olive oil ADVERTISEMENT, 1/4 cup ...",Heat oil in a large pot over medium heat. Add ...
VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Coconut Milk-Free Panang Curry Chicken,"[2 cups light cream ADVERTISEMENT, 1/4 teaspoo...",Heat cream and coconut extract in a skillet or...
Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,Cooked Cold Salad,"[3 tablespoons bacon grease ADVERTISEMENT, 2 c...",Heat bacon grease in a skillet over medium-hig...
ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,Easy Eggnog Creme Brulee,"[4 egg yolks ADVERTISEMENT, 1 tablespoon white...",Preheat oven to 350 degrees F (175 degrees C)....


<class 'pandas.core.frame.DataFrame'>
Index: 39522 entries, rmK12Uau.ntP510KeImX506H6Mr6jTu to 2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         39522 non-null  object
 1   ingredients   39522 non-null  object
 2   instructions  39522 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


# Merge datasets

In [22]:
frames = [data_fn_cln, data_epi_cln, data_ar_cln]
data_all = pd.concat(frames)
display(data_all)
data_all.info()

,title,ingredients,instructions
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,Grammie Hamblet's Deviled Crab,"[1/2 cup celery, finely chopped, 1 small green...",Toss ingredients lightly and spoon into a butt...
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,Infineon Raceway Baked Beans,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Watch how to make this recipe.\nSprinkle the s...
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,Southwestern Black Bean Dip,"[1 1/2 cups dried black beans, picked over and...","In a large saucepan, let the beans soak in eno..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,Sour Cream Noodle Bake,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Watch how to make this recipe.\nPreheat the ov...
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,Sushi Renovation,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Special equipment: sushi mat\nCook the brown r...
...,...,...,...
gehEOcDPtU3SmNSXrwWwWD4ulPpUdMO,Thai-Indian Veggie Soup,"[2 teaspoons olive oil ADVERTISEMENT, 1/4 cup ...",Heat oil in a large pot over medium heat. Add ...
VRAsyF.1xMBYqAVKX1biyIORH6N6qzy,Coconut Milk-Free Panang Curry Chicken,"[2 cups light cream ADVERTISEMENT, 1/4 teaspoo...",Heat cream and coconut extract in a skillet or...
Lf8/u.0k2029QMSQFrHS4gRsvKOQFUG,Cooked Cold Salad,"[3 tablespoons bacon grease ADVERTISEMENT, 2 c...",Heat bacon grease in a skillet over medium-hig...
ay.AqX/9ysBtWHcnHoDeGAyJ5Orla8e,Easy Eggnog Creme Brulee,"[4 egg yolks ADVERTISEMENT, 1 tablespoon white...",Preheat oven to 350 degrees F (175 degrees C)....


<class 'pandas.core.frame.DataFrame'>
Index: 124473 entries, p3pKOD6jIHEcjf20CCXohP8uqkG5dGi to 2Q3Zpfgt/PUwn1YABjJ5A9T3ZW8xwVa
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         124473 non-null  object
 1   ingredients   124473 non-null  object
 2   instructions  124473 non-null  object
dtypes: object(3)
memory usage: 3.8+ MB


In [23]:
data_all.index = range(len(data_all))
data_all.head() 

,title,ingredients,instructions
0,Grammie Hamblet's Deviled Crab,"[1/2 cup celery, finely chopped, 1 small green...",Toss ingredients lightly and spoon into a butt...
1,Infineon Raceway Baked Beans,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Watch how to make this recipe.\nSprinkle the s...
2,Southwestern Black Bean Dip,"[1 1/2 cups dried black beans, picked over and...","In a large saucepan, let the beans soak in eno..."
3,Sour Cream Noodle Bake,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Watch how to make this recipe.\nPreheat the ov...
4,Sushi Renovation,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Special equipment: sushi mat\nCook the brown r...


In [25]:
data_all.to_csv("recipes_all.csv")